# Chatbot implementation

In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")

In [4]:
# Extract data from the PDF
def extract_data_from_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [5]:
extracted_data = extract_data_from_pdf("/Users/tharushavihanga/Developer/Medical_chatbot_using_LLama_2/data")

In [6]:
#extracted_data

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20)
    split_documents = text_splitter.split_documents(extracted_data)
    return split_documents

In [8]:
text_chunks = text_split(extracted_data)

In [9]:
len(text_chunks)

5859

In [10]:
# Download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

/var/folders/h3/dshrqqgs1xv628f0c7bj8mj80000gn/T/ipykernel_53631/745098240.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/opt/anaconda3/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("What is the treatment for diabetes?")
print(len(query_result))

384


In [14]:
# query_result

In [15]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "medicalchatbot"
#index = pc.Index("medicalchatbot")

In [16]:
from langchain_pinecone import Pinecone
from langchain_huggingface import HuggingFaceEmbeddings # Updated import

In [38]:
# Create the LangChain Pinecone vector store from the text chunks
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)

print("LangChain Pinecone vector store created successfully!")

LangChain Pinecone vector store created successfully!


In [17]:
#If we already have an index we can load it like this
docsearch=Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs)

Result [Document(id='70ab2137-2450-4413-9bef-0a7f6f805789', metadata={}, page_content='Purpose\nAllergy is a reaction of the immune system. Nor-\nmally, the immune system responds to foreign microor-\nganisms and particles, like pollen or dust, by producing\nspecific proteins called antibodies that are capable of\nbinding to identifying molecules, or antigens, on the\nforeign organisms. This reaction between antibody and\nantigen sets off a series of reactions designed to protect\nthe body from infection. Sometimes, this same series of'), Document(id='1fec1107-71c2-4fb7-a57b-8cbb0684a42e', metadata={}, page_content='reaction. Allergic rhinitis is characterized by an itchy,\nrunny nose, often with a scratchy or irritated throat due\nto post-nasal drip. Inflammation of the thin membrane\ncovering the eye (allergic conjunctivitis) causes redness,\nirritation, and increased tearing in the eyes. Asthma caus-\nes wheezing, coughing, and shortness of breath. Symp-\ntoms of food allergies depe

In [18]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [19]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [21]:
llm=CTransformers(model="/Users/tharushavihanga/Developer/Medical_chatbot_using_LLama_2/model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':128,
                          'temperature':0.3,
                          'threads': 8})

In [22]:
# llm = HuggingFaceEndpoint(repo_id="mistralai/Mistral-7B-Instruct-v0.3",
#                           temperature=0.7, task="text-generation" ,
#                           huggingfacehub_api_token=HUGGINGFACE_API_KEY)

In [23]:
qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 1}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

/var/folders/h3/dshrqqgs1xv628f0c7bj8mj80000gn/T/ipykernel_53631/4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})


Response :  Acne is a common skin condition characterized by inflammation of oil glands in the skin, leading to redness, swelling, and pimples on the face, chest, and back. It can be caused by hormonal changes, genetics, environmental factors, and certain medications. Acne can have a significant impact on an individual's self-esteem and mental health, and it is important to seek medical attention if symptoms persist or worsen over time.
